# Import packages

In [41]:
import pandas as pd
import re

import sys
sys.path.append('../')
from aqc_utils.gaussian_log_extractor import *
from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

# Objectif :
definir une fonction qui permet de generer un dataframe ou d'updater un .csv avec les molecules calculées et toutes leurs caracteristiques

## First Step
Recupérer les datas d'une molécule

In [42]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


In [43]:
jobs_c = sm.get_jobs(slurm_status.created)
sm.update_status(sm.get_jobs())
sm.update_submitted_jobs()

grep: C11H10O_b7a8_conf_0.log: No such file or directory
grep: C14H18OSi_95fa_conf_0.log: No such file or directory
grep: C16H19NO_f5e7_conf_0.log: No such file or directory
grep: C11H10O2_d7c7_conf_0.log: No such file or directory
grep: C11H10O_9eb4_conf_0.log: No such file or directory
grep: C8H10O_fd16_conf_0.log: No such file or directory
grep: C11H17NO_9080_conf_0.log: No such file or directory
grep: C11H17NO_d167_conf_0.log: No such file or directory
grep: C8H10O2_8f9c_conf_0.log: No such file or directory
grep: C9H12O2_df4c_conf_0.log: No such file or directory
grep: C10H11NO_10ad_conf_0.log: No such file or directory
grep: C14H13NO_5f00_conf_0.log: No such file or directory
grep: C12H12O_d53c_conf_0.log: No such file or directory
grep: C13H14O_eb43_conf_0.log: No such file or directory
grep: C13H16OSi_b5e5_conf_0.log: No such file or directory
grep: C16H22OSi_6b20_conf_0.log: No such file or directory
grep: C14H18OSi_822f_conf_0.log: No such file or directory
grep: C15H20OSi_a8

grep: C23H30O3_9b76_conf_0.log: No such file or directory
grep: C21H19FO2_6f90_conf_0.log: No such file or directory
grep: C20H25NO3_662d_conf_0.log: No such file or directory
grep: C17H18O4_ef6c_conf_0.log: No such file or directory
grep: C24H36O2Si_f68a_conf_0.log: No such file or directory
grep: C15H17NO2_e7ee_conf_0.log: No such file or directory
grep: C15H22O2_1360_conf_0.log: No such file or directory
grep: C12H13F3O2_c0b9_conf_0.log: No such file or directory
grep: C17H25BO4_9b0f_conf_0.log: No such file or directory
grep: C16H22O3_394c_conf_0.log: No such file or directory
grep: C12H16O3_f69c_conf_0.log: No such file or directory
grep: C13H19NO2_4fb7_conf_0.log: No such file or directory
grep: C13H18O2_6664_conf_0.log: No such file or directory
grep: C17H18O2_22f3_conf_0.log: No such file or directory
grep: C12H14O4_b865_conf_0.log: No such file or directory
grep: C17H16O3_7711_conf_0.log: No such file or directory
grep: C18H14O3_19a5_conf_0.log: No such file or directory
grep:

 Entering Gaussian System, Link 0=g09
 Input=C18H19NO2_5d34_conf_0.com


grep: C22H22O3_6f98_conf_0.log: No such file or directory
grep: C21H30O3Si_21fd_conf_0.log: No such file or directory
grep: C21H30O3Si_18a6_conf_0.log: No such file or directory
grep: C21H30O3Si_7ff5_conf_0.log: No such file or directory
grep: C21H30O3Si_af72_conf_0.log: No such file or directory
grep: C11H16O2_4c16_conf_0.log: No such file or directory
grep: C8H10O2_574a_conf_0.log: No such file or directory
grep: C13H12O2_d005_conf_0.log: No such file or directory
grep: C11H15NO2_992e_conf_0.log: No such file or directory
grep: C19H20O2_a9f6_conf_0.log: No such file or directory
grep: C15H23NO2_1e0e_conf_0.log: No such file or directory
grep: C19H18O2_4630_conf_0.log: No such file or directory
grep: C15H16O3_6261_conf_0.log: No such file or directory
grep: C10H13NO2_136f_conf_0.log: No such file or directory
grep: C16H15NO2_c5f1_conf_0.log: No such file or directory
grep: C12H16O2_5e9d_conf_0.log: No such file or directory
grep: C12H16O2_b2e0_conf_0.log: No such file or directory
gre

 Entering Gaussian System, Link 0=g09
 Input=C20H28O2_1cb0_conf_0.com
 Entering Gaussian System, Link 0=g09
 Input=C20H28O2_1cb0_conf_1.com


grep: C15H12O_d275_conf_0.log: No such file or directory
grep: C13H12O3_e27f_conf_0.log: No such file or directory
grep: C9H10O2_444d_conf_0.log: No such file or directory
grep: C14H18N2O2_b0ad_conf_0.log: No such file or directory
grep: C12H14F3NO2_3045_conf_0.log: No such file or directory
grep: C11H14FNO2_0756_conf_0.log: No such file or directory
grep: C15H22N2O3_1f6b_conf_0.log: No such file or directory
grep: C25H35NO4_68df_conf_0.log: No such file or directory
grep: C15H21NO3_6df1_conf_0.log: No such file or directory
grep: C18H20N2O2_5d63_conf_0.log: No such file or directory
grep: C9H11NO2_01e0_conf_0.log: No such file or directory
grep: C11H15NO2_9d06_conf_0.log: No such file or directory
grep: C13H19NO2_23d2_conf_0.log: No such file or directory
grep: C15H21NO4_2e74_conf_0.log: No such file or directory
grep: C11H16N2O2_dc18_conf_0.log: No such file or directory
grep: C14H14O_383b_conf_0.log: No such file or directory
grep: C16H20OSi_5b73_conf_0.log: No such file or director

In [44]:
sm.retrieve_jobs()
sm.get_job_stats(split_by_can=True)

INFO:aqc_utils.slurm_manager:There are 0 running/pending jobs, 5 finished jobs.
INFO:aqc_utils.slurm_manager:Retrieving log files of finished jobs.
INFO:paramiko.transport.sftp:[chan 361] Opened sftp connection (server version 3)


INFO:aqc_utils.gaussian_log_extractor:Output file does not have a 'freq' part. Cannot extract frequencies.


INFO:aqc_utils.slurm_manager:1 jobs finished successfully (all Gaussian steps finished normally). 339 jobs failed.


status,done,failed,incomplete,uploaded
can,,,,
C(Oc1ccccc1)Oc1ccccc1,0,1,0,0
C=CC[Si](Oc1ccc2c(c1)cccc2)(C)C,0,1,0,0
CC(=O)Oc1c2ccccc2cc2c1cccc2,0,1,0,0
CC(=O)Oc1ccc(cc1)C(=O)C,0,1,0,0
CC(=O)Oc1ccc2c(c1)ccc(c2)C(=O)C,0,1,0,0
...,...,...,...,...
OCCOc1ccc(cc1)C,0,1,0,0
OCCOc1ccccc1,0,1,0,0
OCCc1ccc(cc1)OC,0,0,0,1


In [49]:
df = sm.get_job_stats(split_by_can=True)
np.sum(df["failed"])
print(np.sum(df["failed"]), np.sum(df["uploaded"]), np.sum(df["done"])), #np.sum(df["incomplete"]))

338 19 1


(None,)

In [50]:
sm.squeue()

,PENDING,RUNNING
jobs,52,8


# It is very important here to update with a suited tag name the molecules !


In [51]:
sm.upload_done_molecules_to_db(tags="JS_substrate")

INFO:aqc_utils.slurm_manager:There are no molecules with all jobs done. Exitting.


In [52]:
for job in sm.get_jobs(slurm_status.uploaded):
    job_to_change = sm.get_jobs()[job].base_name
    with sm.connection.cd("/home/jschleinitz/gaussian/"):
        try:
            sm.connection.run(f"mv {job_to_change}.chk ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.out ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.log ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.com ../gaussian_uploaded")               
            sm.connection.run(f"mv {job_to_change}.sh ../gaussian_uploaded")
        except:
            pass

mv: cannot stat 'C11H10O_b7a8_conf_0.chk': No such file or directory
mv: cannot stat 'C14H18OSi_95fa_conf_0.chk': No such file or directory
mv: cannot stat 'C16H19NO_f5e7_conf_0.chk': No such file or directory
mv: cannot stat 'C11H10O2_d7c7_conf_0.chk': No such file or directory
mv: cannot stat 'C11H10O_9eb4_conf_0.chk': No such file or directory
mv: cannot stat 'C8H10O_fd16_conf_0.chk': No such file or directory
mv: cannot stat 'C11H17NO_9080_conf_0.chk': No such file or directory
mv: cannot stat 'C11H17NO_d167_conf_0.chk': No such file or directory
mv: cannot stat 'C8H10O2_8f9c_conf_0.chk': No such file or directory
mv: cannot stat 'C9H12O2_df4c_conf_0.chk': No such file or directory
mv: cannot stat 'C10H11NO_10ad_conf_0.chk': No such file or directory
mv: cannot stat 'C14H13NO_5f00_conf_0.chk': No such file or directory
mv: cannot stat 'C12H12O_d53c_conf_0.chk': No such file or directory
mv: cannot stat 'C13H14O_eb43_conf_0.chk': No such file or directory
mv: cannot stat 'C13H16OSi_